<a href="https://colab.research.google.com/github/Abhishek1236/deep-learning-model/blob/master/restricted_boltzman_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving P16-Boltzmann-Machines.zip to P16-Boltzmann-Machines.zip


In [0]:
import zipfile
import io
zf = zipfile.ZipFile(io.BytesIO(uploaded['P16-Boltzmann-Machines.zip']), "r")
zf.extractall()

In [0]:
#importing the libraries
import numpy as np 
import pandas as pd 
import torch 
import torch.nn as nn 
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [0]:
#importing the dataset
movies = pd.read_csv('/content/Boltzmann_Machines/ml-1m/movies.dat',sep = '::',header = None,engine = 'python',encoding = 'latin-1')
users = pd.read_csv('/content/Boltzmann_Machines/ml-1m/users.dat',sep = '::',header = None,engine = 'python',encoding = 'latin-1')
ratings = pd.read_csv('/content/Boltzmann_Machines/ml-1m/ratings.dat',sep = '::',header = None,engine = 'python',encoding = 'latin-1')

In [0]:
#preparing the training and the test sets
training_set = pd.read_csv('/content/Boltzmann_Machines/ml-100k/u1.base',delimiter='\t')
training_set = np.array(training_set,dtype= 'int')
test_set = pd.read_csv('/content/Boltzmann_Machines/ml-100k/u1.test',delimiter = '\t')
test_set = np.array(test_set,dtype='int')

In [0]:
#getting the numbers of users and movies 
nb_users = int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]),max(test_set[:,1])))

In [0]:
#converting the data into array with users in lines and movies in columns
def convert(data):
  new_data = []
  for user in range(1,nb_users + 1):
    id_movies = data[:,1][data[:,0] == user]
    id_ratings = data[:,2][data[:,0] == user]
    ratings = np.zeros(nb_movies)
    ratings[id_movies -1] = id_ratings
    new_data.append(list(ratings))
  return new_data

In [0]:
training_set = convert(training_set)
test_set = convert(test_set)

In [0]:
#converting the data into torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [0]:
#converting the ratings into binary ratings 1(liked) or 0(not liked)
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >=3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >=3] = 1

In [32]:
# Creating the architecture of the Neural Network
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

# Training the RBM
nb_epoch = 50
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: tensor(0.3424)
epoch: 2 loss: tensor(0.2450)
epoch: 3 loss: tensor(0.2465)
epoch: 4 loss: tensor(0.2516)
epoch: 5 loss: tensor(0.2507)
epoch: 6 loss: tensor(0.2479)
epoch: 7 loss: tensor(0.2485)
epoch: 8 loss: tensor(0.2487)
epoch: 9 loss: tensor(0.2496)
epoch: 10 loss: tensor(0.2461)
epoch: 11 loss: tensor(0.2489)
epoch: 12 loss: tensor(0.2466)
epoch: 13 loss: tensor(0.2484)
epoch: 14 loss: tensor(0.2471)
epoch: 15 loss: tensor(0.2470)
epoch: 16 loss: tensor(0.2474)
epoch: 17 loss: tensor(0.2503)
epoch: 18 loss: tensor(0.2455)
epoch: 19 loss: tensor(0.2493)
epoch: 20 loss: tensor(0.2467)
epoch: 21 loss: tensor(0.2458)
epoch: 22 loss: tensor(0.2520)
epoch: 23 loss: tensor(0.2458)
epoch: 24 loss: tensor(0.2472)
epoch: 25 loss: tensor(0.2493)
epoch: 26 loss: tensor(0.2476)
epoch: 27 loss: tensor(0.2473)
epoch: 28 loss: tensor(0.2493)
epoch: 29 loss: tensor(0.2495)
epoch: 30 loss: tensor(0.2466)
epoch: 31 loss: tensor(0.2459)
epoch: 32 loss: tensor(0.2498)
epoch: 33 loss: t

In [33]:
# Testing the RBM
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2407)
